# Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes

path = '../Results/test/'

In [ ]:
# Load config data 
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')

exp_config = multimodal_mazes.load_exp_config(path + 'exp_config.ini')
print(exp_config['noise_scale'])

In [ ]:
# Load data 
x = np.load(path + '99.npy')

top_agent = np.where(x['fitness'] == x['fitness'].max())

with open(path + '99.pickle', 'rb') as file:
    genomes = pickle.load(file)

genome_id, genome, channels = genomes[top_agent[0][0]]

print(x[top_agent[0][0]])
print(genome.size())

In [ ]:
# Plotting

# Fitness 
multimodal_mazes.plot_fitness_over_generations(x, plot_species=True)

# Architecture
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D'}
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

# Filtered architecture 
genome = multimodal_mazes.prune_architecture(genome, config)
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

In [ ]:
# Top genomes 
fitness_cutoff = 1.0

# Generate mazes
maze = multimodal_mazes.TrackMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
maze.generate(exp_config['n_mazes'], noise_scale=exp_config['noise_scale'])

# Test agents 
top_agents = [] 
for g in np.where(x['fitness'] >= fitness_cutoff)[0]: 
    _, genome, channels = genomes[g]
    genome = multimodal_mazes.prune_architecture(genome, config)
    fitness = multimodal_mazes.eval_fitness(genome, config, channels, maze, exp_config['n_steps'])
    if fitness >= fitness_cutoff: 
        top_agents.append(g)

In [ ]:
# WIP: architecture metrics 

top_agents_metrics = []
for g in top_agents:
    _, genome, channels = genomes[g]
    genome = multimodal_mazes.prune_architecture(genome, config)
    arch_metrics = multimodal_mazes.architecture_metrics(genome, config, channels)
    top_agents_metrics.append(list(arch_metrics.values()))
        
top_agents_metrics = np.array(top_agents_metrics)

# Plotting 
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [0.8, 0.2]}, figsize=(10, 5))
a0.plot(top_agents_metrics[:,:-2].T, c='k', linewidth=1.5, alpha=0.25);
a0.set_xticks(np.arange(len(arch_metrics) -2), list(arch_metrics.keys())[:-2], rotation='vertical')
a0.set_ylabel('Number')

a1.plot(top_agents_metrics[:,-2:].T, c='k', linewidth=1.5, alpha=0.25);
a1.set_xticks([0,1], list(arch_metrics.keys())[-2:], rotation='vertical')
a1.set_ylabel('Ratio')
a1.set_xlim([-0.25, 1.25])
a1.set_ylim([-0.025, 1.025])

In [ ]:
# WIP: noise 
noise_scales = np.linspace(start=0.0, stop=0.5, num=10)

noise_results = np.zeros((len(top_agents), len(noise_scales)))

agnt_random_baseline = multimodal_mazes.AgentRandom(location=None, channels=[0,0])
noise_baseline = np.zeros((len(top_agents), len(noise_scales)))

for a, noise_scale in enumerate(noise_scales):
    # Generate mazes 
    maze = multimodal_mazes.TrackMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
    maze.generate(number=100, noise_scale=noise_scale)

    # Test agents 
    for b, g in enumerate(top_agents): 
        _, genome, channels = genomes[g]
        genome = multimodal_mazes.prune_architecture(genome, config)
        noise_results[b,a] = multimodal_mazes.eval_fitness(genome, config, channels, maze, exp_config['n_steps'])

        # Random baseline 
        noise_baseline[b, a] = multimodal_mazes.eval_fitness(genome=None, config=None, channels=[0,0], maze=maze, n_steps=exp_config['n_steps'], agnt=agnt_random_baseline)

    print(a)

noise_results_min = np.argmin(np.trapz(y=noise_results, x=noise_scales))
noise_results_max = np.argmax(np.trapz(y=noise_results, x=noise_scales))

print(noise_results_min, noise_results_max) # 123

In [ ]:
# WIP: noise
plt.plot(noise_scales, noise_results.T, c='k', alpha=0.1);
plt.plot(noise_scales, np.mean(noise_baseline, axis=0), c='xkcd:gray', ls='dotted')
plt.fill_between(noise_scales, np.min(noise_baseline, axis=0), np.max(noise_baseline, axis=0), color='xkcd:gray', alpha=0.25)
plt.ylim(0, 1.0)
plt.xlabel('Noise scale')
plt.ylabel('Fitness')

plt.plot(noise_scales, noise_results[noise_results_min], c='xkcd:purple', alpha=.75);
plt.plot(noise_scales, noise_results[noise_results_max], c='xkcd:dark seafoam', alpha=.75);

In [ ]:
# WIP: noise
for g in [noise_results_min, noise_results_max]: 
    _, genome, channels = genomes[top_agents[g]]
    genome = multimodal_mazes.prune_architecture(genome, config)
    multimodal_mazes.plot_architecture(genome, config, node_names=node_names)